In [50]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import pandas as pd
import numpy as np
import os
import feather
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error 
import random
import csv

In [48]:
import sys
if "/proj/yunligrp/users/minzhi/utils/pylib" not in sys.path:
    sys.path.insert(0, "/proj/yunligrp/users/minzhi/utils/pylib")

In [49]:
from function_process_data_eqtl import *
from function_asso import *
from function_mesa_cca import *
%autoreload 2

In [51]:
def cn_map(cn, map_df, common_col):
    cn_mapped = map_df.merge(cn, on = common_col, how = "inner")
    return cn_mapped

In [52]:
def cn_map_list(cohort_list, cohort_dir_list, map_raw_df, common_col, save_dir):
    cohort_cn_map_summary = pd.DataFrame(columns = ["annotation"], index = cohort_list)
    for cohort_dir_filename, cohort_i in zip(cohort_dir_list, cohort_list):
        cohort = pd.read_csv(cohort_dir_filename, sep = "\t", header = None, index_col = None)
        cohort.rename(columns={0:'NWDID'}, inplace = True)
        map_df = map_raw_df[["NWDID", "unique_subject_key", "subject_id"]]
        map_df.dropna(axis = 0, subset = ["unique_subject_key", "subject_id"], how = "any", inplace = True)
        common_col = "NWDID"
        tmp_cn_mapped = cn_map(cohort, map_df, common_col)
        tmp_cn_mapped_filename = "%s_subject_id_cram.tsv"%cohort_i
        tmp_cn_mapped_dir_filename = os.path.join(save_dir, tmp_cn_mapped_filename)
        tmp_cn_mapped.to_csv(tmp_cn_mapped_dir_filename, sep = "\t", header = True, index = False)
        cohort_cn_map_summary.loc[cohort_i, "annotation"] = tmp_cn_mapped.shape[0]
    cohort_cn_map_summary.sort_index(axis = 0, inplace = True)
    return cohort_cn_map_summary

In [53]:
def split_df(df, cat_col, category_list, file_prefix, save_dir):
    for category in category_list:
        tmp_df = df.loc[df[cat_col] == category, :]
        tmp_filename = "%s_%s.tsv"%(file_prefix, category)
        tmp_dir_filename = os.path.join(save_dir, tmp_filename)
        tmp_df.to_csv(tmp_dir_filename, sep = "\t", header = True, index = False)
    return 1

In [54]:
def compare_df_pair(df1, df2, common_col, df1_col, df2_col):
    df1_df2 = df1.merge(df2, on = common_col, how = "inner")
    overlap_num = df1_df2.shape[0]
    df1_num = df1.shape[0]
    df2_num = df2.shape[0]
    df1_nodf2_num = df1_num - overlap_num
    df2_nodf1_num = df2_num - overlap_num
    if df1_col == df2_col:
        df1_col_x = "%s_x"%df1_col
        df2_col_y = "%s_y"%df2_col
    if (df1[df1_col].dtypes == "float64" or df1[df1_col].dtypes == "int32") and (df2[df2_col].dtypes == "float64" or df2[df2_col].dtypes == "int32"):
        cor = pearsonr(df1_df2[df1_col_x], df1_df2[df2_col_y])[0]
        return df1_nodf2_num, df2_nodf1_num, overlap_num, df1_num, df2_num, cor
    else:
        return df1_nodf2_num, df2_nodf1_num, overlap_num, df1_num, df2_num

In [55]:
def df_extraction_duplicates(df, col):
    boolean_series = df[[col]].duplicated(keep = False)
    duplicated_df = df.loc[boolean_series, :]
    return duplicated_df

In [56]:
def concat_egfr(egfr_dir, cohort_list, header_selection):
    egfr_list = []
    for cohort in cohort_list:
        egfr_dir_filename = os.path.join(egfr_dir, "egfr_calculated_%s.tsv"%cohort)
        egfr_raw = pd.read_csv(egfr_dir_filename, sep = "\t", header = 0, index_col = None)
        egfr = egfr_raw[header_selection]
        cohort_pheno_table = "_".join(cohort.split("-"))
        egfr["cohort"] = egfr.shape[0] * [cohort_pheno_table]
        egfr["unique_subject_key"] = egfr.shape[0] * [None]
        egfr_num = egfr.shape[0]
        for egfr_i in range(egfr_num):
            egfr.loc[egfr_i, "unique_subject_key"] = "%s_%s"%(cohort_pheno_table, egfr.loc[egfr_i, "id"])
        egfr.rename(columns = {"id":"SUBJECT_ID", "age":"age_at_EGFRCKDEPI", "EGFR":"EGFRCKDEPI"}, inplace = True)
        egfr_list.append(egfr)
        print("%s completed."%cohort)
    egfr_full = pd.concat(egfr_list, axis = 0)
    egfr_full.reset_index(drop = True, inplace = True)
    return egfr_full

In [57]:
def check_invar(df, col):
    check_invar_list = df[col].values.tolist()
    if len(set(check_invar_list)) == 1:
        invar_bool = True
    else:
        invar_bool = False
    return invar_bool

def check_list_invar(df, col_list):
    invar_list = []
    for col in col_list:
        invar_bool = check_invar(df, col)
        invar_list.append(invar_bool)
    return invar_list

In [58]:
def remove_dup_anno(annotation, invar_subsets, rm_header = None):
    if rm_header == None:
        annotation["aux_rm"] = np.arange(annotation.shape[0])
        rm_header = "aux_rm"
    annotation_dup = df_extraction_duplicates(annotation, "unique_subject_key")
    _, usk_list = categorize_df(annotation_dup, "unique_subject_key")
    remove_header_list = []
    for usk in usk_list:
        tmp_annotation = annotation_dup.loc[annotation_dup.loc[:, "unique_subject_key"] == usk, :]
        tmp_invar_list = check_list_invar(tmp_annotation, invar_subsets)
        tmp_rm_list = tmp_annotation[rm_header].values.tolist()
        if False in tmp_invar_list:
            remove_header_list = remove_header_list + tmp_rm_list
        else:
            tmp_rm_num = len(tmp_rm_list)
            selection = random.randint(0, tmp_rm_num - 1)
            remove_header_list = remove_header_list + [tmp_rm_list[selection]]
    annotation_unique = annotation[~annotation[rm_header].isin(remove_header_list)]
    if rm_header == "aux_rm":
        annotation_unique.drop(axis = 1, labels = ["aux_rm"], inplace = True)
    return annotation_unique

In [59]:
def merge_replace_nan(df0_col, df1_col, df0, df1):
    df0 = df0.set_index(df0_col)
    df1 = df1.set_index(df1_col)
    df0_filled = df0.fillna(df1)
    df0_filled = df0_filled.reset_index()
    return df0_filled

In [60]:
def func(x):
    if x.first_valid_index() is None:
        return None
    else:
        return x[x.first_valid_index()]

In [61]:
def map_annotation(pheno, annotation, pheno_col, annotation_col, pivot_col, how_merge, pheno_prefix, anno_prefix, mapped_save_dir, diff_save_dir):
    pheno_mapped = annotation.merge(pheno, left_on = annotation_col[0], right_on = pheno_col[0], how = merge_how)
    col_num = len(pheno_col[1:])
    for col_i in range(1, col_num + 1):
        if pheno_col[col_i] != annotation_col[col_i]:
            pheno_mapped.drop(axis = 1, columns = [pheno_col[col_i], annotation_col[col_i]], inplace = True)
        else:
            tmp_pheno_col = "%s_y"%pheno_col[col_i]
            tmp_annotation_col = "%s_x"%annotation_col[col_i]
            pheno_mapped.drop(axis = 1, columns = [tmp_pheno_col, tmp_annotation_col], inplace = True)
    if len(annotation_col) > 1:
        depre_sample_list, pivot_col_df = map_deprecation_list(pheno, annotation, pheno_col, annotation_col,
                                                               pivot_col, pheno_prefix, anno_prefix, diff_save_dir)
        if depre_sample_list != []:
            pheno_mapped_raw = pheno_mapped.copy()
            del pheno_mapped
            pheno_mapped = pheno_mapped_raw[~pheno_mapped_raw[pivot_col].isin(depre_sample_list)]
        pheno_mapped = pheno_mapped.merge(pivot_col_df, on = pivot_col, how = "inner")
    pheno_mapped_filename = "%s_%s.tsv"%(anno_prefix, pheno_prefix)
    pheno_mapped_dir_filename = os.path.join(mapped_save_dir, pheno_mapped_filename)
    pheno_mapped.to_csv(pheno_mapped_dir_filename, sep = "\t", header = True, index = False)
    return pheno_mapped

In [62]:
def map_deprecation_list(pheno, annotation, pheno_col, annotation_col, pivot_col, pheno_prefix, anno_prefix, save_dir):
    pheno_annotatio_dif = annotation.merge(pheno, left_on = pheno_col[0], right_on = annotation_col[0], how = "outer")
    depre_sample_list = []
    pivot_df = pheno_annotatio_dif[[pivot_col]]
    for pheno_col_i, annotation_col_i in zip(pheno_col[1:], annotation_col[1:]):
        if pheno_col_i == annotation_col_i:
            annotation_col_i = "%s_x"%annotation_col_i
            pheno_col_i = "%s_y"%pheno_col_i
        col_dif = pheno_annotatio_dif[[pivot_col, annotation_col_i, pheno_col_i]]
        tmp_depre_sample_list, df_col_i = map_deprecation(col_dif, pheno_col_i, annotation_col_i,
                                                          pivot_col, pheno_prefix, anno_prefix, save_dir)
        pivot_df = pivot_df.merge(df_col_i, on = pivot_col, how = "inner")
        depre_sample_list = depre_sample_list + tmp_depre_sample_list
    return depre_sample_list, pivot_df

In [63]:
def map_deprecation(df, pheno_col, annotation_col, pivot_col, pheno_prefix, anno_prefix, save_dir):
    df_pivot = df[[pivot_col]]
    df_col = df[[pheno_col, annotation_col]]
    df_col.iloc[:, 0] = df_col.apply(func, axis = 1)
    df_col.iloc[:, 1] = df_col.apply(func, axis = 1)
    del df
    df = pd.concat([df_pivot, df_col], axis = 1)
    depre_sample_index_list = df[df.iloc[:, 0].isnull()].index.tolist()
    if depre_sample_index_list == []:
        depre_sample_list = []
    else:
        depre_sample_list = df.loc[depre_sample_index_list, pivot_col].values.reshape(1, -1).tolist()[0]
    df.dropna(axis = 0, how = "any", inplace = True)
    tmp_compare = df[pheno_col].eq(df[annotation_col], axis = 0)
    df_dif = df[tmp_compare == False]
    df_cons = df[tmp_compare == True]
    if annotation_col.split('_x')[-1] == "":
        annotation_col_propagate = annotation_col.split('_x')[0]
        df_cons.rename(columns = {annotation_col:annotation_col_propagate}, inplace = True)
    else:
        annotation_col_propagate = annotation_col
    df_cons_propagate = df_cons.loc[:, [pivot_col, annotation_col_propagate]]
    if df_dif.shape[0] != 0:
        print("%s is inconsistent."%pheno_col)
        df_dif_filename = "%s_%s_%s.tsv"%(anno_prefix, pheno_prefix, annotation_col)
        df_dif_dir_filename = os.path.join(save_dir, df_dif_filename)
        df_dif.to_csv(df_dif_dir_filename, sep = "\t", header = True, index = False)
        depre_sample_list = df_dif[[pivot_col]].values.reshape(1, -1).tolist()[0]
    return depre_sample_list, df_cons_propagate

Pipeline:
1. Annotation File:\
(1) Including exclusion, and only keep the columns we need.\
(2) Check if duplication happened in the "unique_subject_key" and remove: a. anyone of the duplication if they are exactly the same; b. all of them if anyone has anything being different with others.\
(3) Removing the duplicated individuals based on the duplicates list provided by dbGaP or other organizations.
2. Phenotype File:\
(1) Concatenating all phenotype file together -- including preprocessing all phenotype files and concatenating them;\
(2) Converting to get microcytosis and anemia;\
(3) 

## Annotation Side

### Concatenate DDIMER to Annotation File

In [38]:
ddimer_dir = os.path.join("..", "raw_data", "phenotype")
ddimer_filename = "DDIMER_21MAY2019_complete_useful_col.tsv"
ddimer_dir_filename = os.path.join(ddimer_dir, ddimer_filename)
ddimer = pd.read_csv(ddimer_dir_filename, sep = "\t", header = 0, index_col = None)

annotation_dir = os.path.join("..", "raw_data", "annotation")
annotation_filename = "freeze8_anno05_af02_unique02.tsv"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)

In [39]:
pheno_col = ["NWDID", "sex", "cohort"]
annotation_col = ["NWDID", "sex", "study"]
pheno_prefix = "ddimer"
anno_prefix = "freeze8_anno05_af02_unique02"
mapped_save_dir = os.path.join("..", "raw_data", "annotation")
diff_save_dir = os.path.join("..", "data_summary")
pivot_col = "NWDID"
merge_how = "left"
ddimer_mapped = map_annotation(ddimer, annotation, pheno_col, annotation_col, pivot_col, merge_how,
                               pheno_prefix, anno_prefix, mapped_save_dir, diff_save_dir)

## Phenotype Side

### Get btc01-coh_adad01-noex
Concatenate gengrp6, weight and center to btc-coh

In [24]:
btc_dir = os.path.join("..", "raw_data", "phenotype")
btc_filename = "btc01-coh04.tsv"
btc_dir_filename = os.path.join(btc_dir, btc_filename)
btc = pd.read_csv(btc_dir_filename, sep = "\t", header = 0, index_col = None, dtype = {"SUBJECT_ID":"string"})
print(btc.shape)
print("duplicated unique_subject_key", duplicates_num(btc, "unique_subject_key"))

gengrp6_weight_center_dir = os.path.join("..", "raw_data", "phenotype")
gengrp6_weight_center_filename = "gengrp6_center_weight_noex.tsv"
gengrp6_weight_center_dir_filename = os.path.join(gengrp6_weight_center_dir, gengrp6_weight_center_filename)
gengrp6_weight_center = pd.read_csv(gengrp6_weight_center_dir_filename, sep = "\t", header = 0, index_col = None, dtype = {"SUBJECT_ID":"string"})
print("duplicated unique_subject_key", duplicates_num(gengrp6_weight_center, "unique_subject_key"))

(217832, 35)
duplicated unique_subject_key 0
duplicated unique_subject_key 0


In [25]:
# Here we first check if there is confliction between btc and gengrp6_weight_center in the columns "unique_subject_key", "cohort", "SUBJECT_ID"
pheno_col = ["unique_subject_key", "SUBJECT_ID", "cohort"]
annotation_col = ["unique_subject_key", "SUBJECT_ID", "cohort"]
pheno_prefix = "adad01-noex"
anno_prefix = "btc01-coh04"
pivot_col = "unique_subject_key"
merge_how = "left"
mapped_save_dir = os.path.join("..", "raw_data", "phenotype")
diff_save_dir = os.path.join("..", "data_summary")
btc_adad = map_annotation(gengrp6_weight_center, btc, pheno_col, annotation_col, pivot_col, merge_how,
                          pheno_prefix, anno_prefix, mapped_save_dir, diff_save_dir)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


### Get btc01-coh_adad01-noex_egfr
Concatenate egfr to btc01-coh_adad01-noex

#### Dropping useless columns from egfr

In [32]:
egfr_dir = os.path.join("..", "raw_data", "phenotype", "egfr_calculated")
egfr_filename = "egfr03_unique.tsv"
egfr_dir_filename = os.path.join(egfr_dir, egfr_filename)
egfr = pd.read_csv(egfr_dir_filename, sep = "\t", header = 0, index_col = None, dtype = {"SUBJECT_ID":"string"})

In [33]:
egfr_no_race_ethnicity = egfr.drop(columns = ["race", "ethnicity"], inplace = False)
egfr_no_race_ethnicity_dir = os.path.join("..", "raw_data", "phenotype", "egfr_calculated")
egfr_no_race_ethnicity_filename = "egfr03_unique_no_race_ethnicity.tsv"
egfr_no_race_ethnicity_dir_filename = os.path.join(egfr_no_race_ethnicity_dir, egfr_no_race_ethnicity_filename)
egfr_no_race_ethnicity.to_csv(egfr_no_race_ethnicity_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

#### Concatenate egfr to btc01-coh_adad01-noex

In [34]:
egfr_dir = os.path.join("..", "raw_data", "phenotype", "egfr_calculated")
egfr_filename = "egfr03_unique_no_race_ethnicity.tsv"
egfr_dir_filename = os.path.join(egfr_dir, egfr_filename)
egfr = pd.read_csv(egfr_dir_filename, sep = "\t", header = 0, index_col = None, dtype = {"SUBJECT_ID":"string"})

btc_adad_dir = os.path.join("..", "raw_data", "phenotype")
btc_adad_filename = "btc01-coh04_adad01_noex.tsv"
btc_adad_dir_filename = os.path.join(btc_adad_dir, btc_adad_filename)
btc_adad = pd.read_csv(btc_adad_dir_filename, sep = "\t", header = 0, index_col = None, dtype = {"SUBJECT_ID":"string"})

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (34,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
# Here we first check if there is confliction between btc_adad and egfr03 in the columns "unique_subject_key", "cohort", "SUBJECT_ID"
pheno_col = ["unique_subject_key", "SUBJECT_ID", "cohort"]
annotation_col = ["unique_subject_key", "SUBJECT_ID", "cohort"]
pheno_prefix = "egfr03"
anno_prefix = "btc01-coh04_adad01-noex"
mapped_save_dir = os.path.join("..", "raw_data", "phenotype")
diff_save_dir = os.path.join("..", "data_summary")
pivot_col = "unique_subject_key"
merge_how = "outer"
btc_adad_egfr = map_annotation(egfr, btc_adad, pheno_col, annotation_col, pivot_col, merge_how,
                             pheno_prefix, anno_prefix, mapped_save_dir, diff_save_dir)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


### Get btc03-coh_adad01-noex_egfr-ckd
Adding CKD, microcytosis, and anemia

In [36]:
btc_adad_egfr_ckd = one_condition_conversion(btc_adad_egfr, "EGFRCKDEPI", 60, "CKD")
btc_microcytosis_adad_egfr_ckd = one_condition_conversion(btc_adad_egfr_ckd, "mcv_entvol_rbc_1", 80, "microcytosis")
btc02_adad_egfr_ckd = two_condition_conversion(btc_microcytosis_adad_egfr_ckd, "hemoglobin_mcnc_bld_1", 13, 12, "male", "anemia")

In [37]:
btc02_adad_egfr_ckd_dir = os.path.join("..", "raw_data", "phenotype")
btc02_adad_egfr_ckd_filename = "btc02-coh04_adad01-noex_egfr03-ckd.tsv"
btc02_adad_egfr_ckd_dir_filename = os.path.join(btc02_adad_egfr_ckd_dir, btc02_adad_egfr_ckd_filename)
btc02_adad_egfr_ckd.to_csv(btc02_adad_egfr_ckd_dir_filename, sep = "\t", header = True, index = False)

## Combining Annotation and Phenotype Sides

### Concatenate Annotation (w DDIMER) and BCT-ADAD01_eGFR

In [41]:
btc_adad_egfr_dir = os.path.join("..", "raw_data", "phenotype")
btc_adad_egfr_filename = "btc02-coh04_adad01-noex_egfr03-ckd.tsv"
btc_adad_egfr_dir_filename = os.path.join(btc_adad_egfr_dir, btc_adad_egfr_filename)
btc_adad_egfr = pd.read_csv(btc_adad_egfr_dir_filename, sep = "\t", header = 0, index_col = None)

anno_ddimer_dir = os.path.join("..", "raw_data", "annotation")
anno_ddimer_filename = "freeze8_anno05_af02_unique02_ddimer.tsv"
anno_ddimer_dir_filename = os.path.join(anno_ddimer_dir, anno_ddimer_filename)
anno_ddimer = pd.read_csv(anno_ddimer_dir_filename, sep = "\t", header = 0, index_col = None)

In [42]:
# Here we first check if there is confliction between btc_adad and egfr03 in the columns "unique_subject_key", "cohort", "SUBJECT_ID"
pheno_col = ["unique_subject_key", "SUBJECT_ID", "cohort", "male"]
annotation_col = ["unique_subject_key", "subject_id", "study", "sex"]
pheno_prefix = "btc02-coh04_ddimer-noex_egfr03-ckd_adad01-noex"
anno_prefix = "freeze8_anno05_af02"
mapped_save_dir = os.path.join("..", "prepro_data", "phenotype")
diff_save_dir = os.path.join("..", "data_summary")
pivot_col = "unique_subject_key"
merge_how = "inner"
anno_ddimer_btc_adad_egfr = map_annotation(btc_adad_egfr, anno_ddimer, pheno_col, annotation_col, pivot_col, merge_how,
                                           pheno_prefix, anno_prefix, mapped_save_dir, diff_save_dir)

### Applying Exclusion Strategy on Mapped Phenotypes

In [18]:
pheno_noex_adad_noex_dir = os.path.join("..", "prepro_data", "phenotype")
pheno_noex_adad_noex_filename = "freeze8_anno05_af02_btc02-coh04_ddimer-noex_egfr03-ckd_adad01-noex.tsv"
pheno_noex_adad_noex_dir_filename = os.path.join(pheno_noex_adad_noex_dir, pheno_noex_adad_noex_filename)
pheno_noex_adad_noex = pd.read_csv(pheno_noex_adad_noex_dir_filename, sep = "\t", header = 0, index_col = None)

#### Gengrp6: excluding samples with CONSENT_text == DROP no matter how INTERNAL_USE_ONLY is

In [19]:
print(pheno_noex_adad_noex.shape)
pheno_noex_adad = pheno_noex_adad_noex.loc[pheno_noex_adad_noex.loc[:, "CONSENT_text"] != "DROP", :]
print(pheno_noex_adad.shape)

(70594, 56)
(70594, 56)


#### DDIMER: excluding samples with sample_remove_DDIMER == TRUE

In [20]:
pheno_adad = pheno_noex_adad.loc[pheno_noex_adad.loc[:, "sample_remove_DDIMER"] != 1, :]
print(pheno_adad.shape)

(70594, 56)


#### Saving Result

In [21]:
pheno_adad_dir = os.path.join("..", "prepro_data", "phenotype")
pheno_adad_filename = "freeze8_anno05_af02_btc03-coh04_egfr03-ckd_adad01.tsv"
pheno_adad_dir_filename = os.path.join(pheno_adad_dir, pheno_adad_filename)
pheno_adad.to_csv(pheno_adad_dir_filename, sep = "\t", header = True, index = False)

## Appending Race and Ethnicity to Annotation & Phenotype File

In [12]:
def assign_uniform_race_ethnicity(df, cohort_col, cohort, race, ethnicity, race_col, ethnicity_col):
    df.loc[df.loc[:, cohort_col] == cohort, race_col] = race
    df.loc[df.loc[:, cohort_col] == cohort, ethnicity_col] = ethnicity
    return df

In [30]:
anno_pheno_dir = os.path.join("..", "prepro_data", "phenotype")
anno_pheno_filename = "freeze8_anno05_af02_btc03-coh04_egfr03-ckd_adad01.tsv"
anno_pheno_dir_filename = os.path.join(anno_pheno_dir, anno_pheno_filename)
anno_pheno = pd.read_csv(anno_pheno_dir_filename, sep = "\t", header = 0, index_col = None)

In [31]:
race_dict = {"hispanic":0, "white":1, "black":2, "asian":3, "AI_AN":4, "HI_PI":5, "other":6, "multiple":7}
ethnicity_dict = {"non-hispanic":0, "hispanic":1}
race_col = "race"
ethnicity_col = "ethnicity"
anno_pheno[race_col] = np.nan
anno_pheno[ethnicity_col] = np.nan
cohort_race_ethnicity_dict = {"FHS":{"race":"white", "ethnicity":"non-hispanic"}, "Amish":{"race":"white", "ethnicity":"non-hispanic"},
                              "GenSalt":{"race":"asian", "ethnicity":"non-hispanic"}, "HyperGEN":{"race":"black", "ethnicity":"non-hispanic"},
                              "DHS":{"race":"black", "ethnicity":"non-hispanic"}, "GENOA":{"race":"black", "ethnicity":"non-hispanic"},
                              "JHS":{"race":"black", "ethnicity":"non-hispanic"}, "SAFS":{"race":"hispanic", "ethnicity":"hispanic"}
                             }
cohort_col = "study"

In [32]:
for cohort in cohort_race_ethnicity_dict:
    race_ethnicity_dict = cohort_race_ethnicity_dict[cohort]
    race_name = race_ethnicity_dict[race_col]
    ethnicity_name = race_ethnicity_dict[ethnicity_col]
    race = race_dict[race_name]
    ethnicity = ethnicity_dict[ethnicity_name]
    anno_pheno = assign_uniform_race_ethnicity(anno_pheno, cohort_col, cohort, race, ethnicity, race_col, ethnicity_col)
    print(cohort)

FHS
Amish
GenSalt
HyperGEN
DHS
GENOA
JHS
SAFS


In [33]:
race_ethnicity_dir = os.path.join("..", "raw_data", "race_ethnicity")
race_ethnicity_filename = "race_ethnicity.tsv"
race_ethnicity_dir_filename = os.path.join(race_ethnicity_dir, race_ethnicity_filename)
race_ethnicity = pd.read_csv(race_ethnicity_dir_filename, sep = "\t", header = 0, index_col = None)

In [34]:
df1 = anno_pheno.reset_index(drop = True)
df2 = race_ethnicity
common_col = ["unique_subject_key"]
fillin_col_raw = list(df2)
fillin_col = [x for x in fillin_col_raw if x not in common_col]
df1_fillin = df1.combine_first(df1.drop(columns = fillin_col).merge(df2, how = 'left', on = common_col))
df1_fillin_dir = os.path.join("..", "prepro_data", "phenotype")
df1_fillin_filename = "freeze8_anno05_af02_btc03-coh04_egfr03-ckd_adad01_race_ethnicity.tsv"
df1_fillin_dir_filename = os.path.join(df1_fillin_dir, df1_fillin_filename)
df1_fillin.to_csv(df1_fillin_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

## Select AA and HL

In [38]:
anno_pheno_re_dir = os.path.join("..", "prepro_data", "phenotype")
anno_pheno_re_filename = "freeze8_anno05_af02_btc03-coh04_egfr03-ckd_adad01_race_ethnicity.tsv"
anno_pheno_re_dir_filename = os.path.join(anno_pheno_re_dir, anno_pheno_re_filename)
anno_pheno_re = pd.read_csv(anno_pheno_re_dir_filename, sep = "\t", header = 0, index_col = None)

In [40]:
anno_pheno_re_aa = anno_pheno_re.copy()
anno_pheno_re_aa["AA"] = np.nan
anno_pheno_re_aa.loc[anno_pheno_re_aa.loc[:, "race"] == 2, "AA"] = 1
anno_pheno_re_aa.loc[anno_pheno_re_aa.loc[:, "race"] != 2, "AA"] = 0
anno_pheno_re_aa["AA_ethnicity"] = anno_pheno_re_aa["AA"] + anno_pheno_re_aa["ethnicity"]
# anno_pheno_re_aa_select = anno_pheno_re_aa.loc[anno_pheno_re_aa.loc[:, "AA_ethnicity"] != 0, :]
anno_pheno_re_aa_select_1 = anno_pheno_re_aa.loc[anno_pheno_re_aa.loc[:, "AA_ethnicity"] == 1, :]
anno_pheno_re_aa_select_2 = anno_pheno_re_aa.loc[anno_pheno_re_aa.loc[:, "AA_ethnicity"] == 2, :]
anno_pheno_re_aa_select = pd.concat([anno_pheno_re_aa_select_1, anno_pheno_re_aa_select_2], axis = 0)
# display(anno_pheno_re_aa_select)
anno_pheno_re_aa_select.drop(columns = ["race", "AA_ethnicity"], inplace = True)
anno_pheno_re_aa_select_dir = os.path.join("..", "prepro_data", "phenotype")
anno_pheno_re_aa_select_filename = "freeze8_anno05_af02_btc03-coh04_egfr03-ckd_adad01_race_ethnicity_0-2.tsv"
anno_pheno_re_aa_select_dir_filename = os.path.join(anno_pheno_re_aa_select_dir, anno_pheno_re_aa_select_filename)
anno_pheno_re_aa_select.to_csv(anno_pheno_re_aa_select_dir_filename, sep = "\t", header = True, index = False)

### Compare Race and Ethnicity b/w from DCC and from eGFR 

In [41]:
pheno_dir = os.path.join("..", "prepro_data", "phenotype")
pheno_filename = "freeze8_anno05_af02_btc03-coh04_egfr03-ckd_adad01_race_ethnicity_0-2.tsv"
pheno_dir_filename = os.path.join(pheno_dir, pheno_filename)
pheno = pd.read_csv(pheno_dir_filename, sep = "\t", header = 0, index_col = None)

In [42]:
egfr_dir = os.path.join("..", "prepro_data", "phenotype")
egfr_filename = "freeze8_anno05_af02_unique02_egfr-ckd.tsv"
egfr_dir_filename = os.path.join(egfr_dir, egfr_filename)
egfr = pd.read_csv(egfr_dir_filename, sep = "\t", header = 0, index_col = None)

In [45]:
pheno_ra = pheno[["NWDID", "unique_subject_key", "AA", "ethnicity"]]
egfr_ra = egfr[["NWDID", "unique_subject_key", "AA", "ethnicity"]]

In [77]:
display(egfr_ra)

,NWDID,unique_subject_key,AA,ethnicity
0,NWD112649,HCHS_SOL_SoL809397,0,1
1,NWD278543,HCHS_SOL_SoL129930,0,1
2,NWD579469,HCHS_SOL_SoL379011,0,1
3,NWD784564,HCHS_SOL_SoL818760,0,1
4,NWD777183,HCHS_SOL_SoL220081,0,1
...,...,...,...,...
23882,NWD999719,HCHS_SOL_SoL943901,0,1
23883,NWD999746,JHS_3539,1,0
23884,NWD999812,HCHS_SOL_SoL239973,0,1
23885,NWD999830,HCHS_SOL_SoL943941,0,1


In [78]:
# Here we first check if there is confliction between btc_adad and egfr03 in the columns "unique_subject_key", "cohort", "SUBJECT_ID"
pheno_col = ["unique_subject_key", "NWDID", "AA", "ethnicity"]
annotation_col = ["unique_subject_key", "NWDID", "AA", "ethnicity"]
pheno_prefix = "pheno"
anno_prefix = "egfr"
mapped_save_dir = os.path.join("..", "prepro_data", "phenotype")
diff_save_dir = os.path.join("..", "data_summary")
pivot_col = "unique_subject_key"
merge_how = "inner"
pheno_egfr = map_annotation(pheno_ra, egfr_ra, pheno_col, annotation_col, pivot_col, merge_how,
                            pheno_prefix, anno_prefix, mapped_save_dir, diff_save_dir)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [79]:
display(pheno_egfr)

,unique_subject_key,NWDID,AA,ethnicity
0,HCHS_SOL_SoL809397,NWD112649,0.0,1.0
1,HCHS_SOL_SoL129930,NWD278543,0.0,1.0
2,HCHS_SOL_SoL379011,NWD579469,0.0,1.0
3,HCHS_SOL_SoL818760,NWD784564,0.0,1.0
4,HCHS_SOL_SoL220081,NWD777183,0.0,1.0
...,...,...,...,...
20366,HCHS_SOL_SoL943901,NWD999719,0.0,1.0
20367,JHS_3539,NWD999746,1.0,0.0
20368,HCHS_SOL_SoL239973,NWD999812,0.0,1.0
20369,HCHS_SOL_SoL943941,NWD999830,0.0,1.0


## CN Side

### CN of Each Cohort Based on Phenotype

In [28]:
pheno_dir = os.path.join("..", "prepro_data", "phenotype")
pheno_filename = "freeze8_anno05_af02_btc03-coh04_egfr03-ckd_adad01_race_ethnicity_0-2.tsv"
pheno_dir_filename = os.path.join(pheno_dir, pheno_filename)
pheno = pd.read_csv(pheno_dir_filename, sep = "\t", header = 0, index_col = None)
_, pheno_cohort_list = categorize_df(pheno, "study")

full_cn_dir = os.path.join("..", "prepro_data", "cn")
full_cn_filename = "alpha_globin_calls_pass_useful.tsv"
full_cn_dir_filename = os.path.join(full_cn_dir, full_cn_filename)
full_cn = pd.read_csv(full_cn_dir_filename, sep = "\t", header = 0, index_col = None)

In [29]:
for cohort in pheno_cohort_list:
    pheno_cohort_raw = pheno.loc[pheno.loc[:, "study"] == cohort, :]
    pheno_id = pheno_cohort_raw[["NWDID"]]
    cn = full_cn.merge(pheno_id, on = "NWDID", how = "inner")
    
    full_cn_id = full_cn[["NWDID"]]
    pheno_cohort = full_cn_id.merge(pheno_cohort_raw, on = "NWDID", how = "inner")
    print(cohort, cn.shape)
    cn_filename = "cn_%s.tsv"%cohort
    cn_dir = os.path.join("..", "cn", cohort, "pre_data")
    cn_dir_filename = os.path.join(cn_dir, cn_filename)
    if not os.path.isdir(cn_dir):
        os.makedirs(cn_dir, exist_ok = True) 
    cn.to_csv(cn_dir_filename, "\t", header = True, index = False)
    
    pheno_cohort_dir = os.path.join("..", "cn", cohort, "pre_data")
    pheno_cohort_filename = f"freeze8_anno05_af02_btc03-coh04_egfr03-ckd_adad01_race_ethnicity_0-2_{cohort}.tsv"
    pheno_cohort_dir_filename = os.path.join(pheno_cohort_dir, pheno_cohort_filename)
    if not os.path.isdir(pheno_cohort_dir):
        os.makedirs(pheno_cohort_dir, exist_ok = True)
    pheno_cohort.to_csv(pheno_cohort_dir_filename, sep = "\t", header = True, index = False)

WHI (1447, 2)
COPDGene (1862, 2)
BioMe (5448, 2)
JHS (3099, 2)
MESA (2106, 2)
ARIC (365, 2)
SAFS (1107, 2)
HyperGEN (1786, 2)
GENOA (1058, 2)
CHS (690, 2)
DHS (372, 2)
GeneSTAR (692, 2)
CARDIA (1354, 2)
HCHS_SOL (3847, 2)


## Appendix B. Preprocess weight, center and gengrp6

In [63]:
gengrp6_filename = "page-harmonized-phenotypes-pca-freeze2-candidate2-2016-12-14.GWASid_fid_22May2018internalPCs.SOLv2consent.txt"
gengrp6_dir = os.path.join("..", "raw_data", "adjustment", "gengrp6")
gengrp6_dir_filename = os.path.join(gengrp6_dir, gengrp6_filename)
gengrp6 = pd.read_csv(gengrp6_dir_filename, sep = "\t", header = 0, index_col = None)
gengrp6.replace(".", np.nan, inplace=True)
gengrp6_select = gengrp6[["z_sol_id", "analysis_id", "CONSENT_text", "INTERNAL_USE_ONLY",
                          "gengrp6", "study"]].dropna(axis=0, subset=["analysis_id","gengrp6"],how="any")
print(gengrp6_select.shape)

gengrp6_select_dropna = gengrp6[["z_sol_id", "analysis_id", "CONSENT_text", "INTERNAL_USE_ONLY", "gengrp6", "study"]].dropna(axis=0, how="any")
gengrp6_select_dropna.rename(columns = {"analysis_id":"SUBJECT_ID", "study":"cohort"}, inplace=True)
print(gengrp6_select_dropna.shape)
cat_col = "cohort"
gengrp6_category_summary, category_list = categorize_df(gengrp6_select_dropna, cat_col)
display(gengrp6_category_summary)
gengrp6_select_dropna["cohort"] = gengrp6_select_dropna.shape[0] * ["HCHS_SOL"]
gengrp6_select_dropna.reset_index(drop = True, inplace = True)

(11829, 6)
(11829, 6)


,cohort,sample_size
0,SOL,11829


In [71]:
gengrp6_select_dropna_num = gengrp6_select_dropna.shape[0]
gengrp6_select_dropna["unique_subject_key"] = gengrp6_select_dropna_num * [None]
for gengrp6_select_dropna_i in range(gengrp6_select_dropna_num):
    gengrp6_select_dropna.loc[gengrp6_select_dropna_i, "unique_subject_key"] = "%s_%s"%("HCHS_SOL", gengrp6_select_dropna.loc[gengrp6_select_dropna_i, "SUBJECT_ID"])
#    if (gengrp6_select_dropna_i + 1) % 100 == 0:
#        print("%d/%d"%(gengrp6_select_dropna_i + 1, gengrp6_select_dropna_num))
gengrp6_select_dropna_dir = os.path.join("..", "raw_data", "phenotype")
gengrp6_select_dropna_filename = "gengrp6_sol_noexclude.tsv"
gengrp6_select_dropna_dir_filename = os.path.join(gengrp6_select_dropna_dir, gengrp6_select_dropna_filename)
gengrp6_select_dropna.to_csv(gengrp6_select_dropna_dir_filename, sep = "\t", header = True, index = False)

weight and center

In [72]:
weight_center_dir = os.path.join("..", "raw_data", "adjustment")
weight_center_filename = "bloodcell_output.csv"
weight_center_dir_filename = os.path.join(weight_center_dir, weight_center_filename)
weight_center = pd.read_csv(weight_center_dir_filename, sep = ",", header = 0, index_col = None)
weight_center_select = weight_center[["ID", "WEIGHT_FINAL_NORM_OVERALL", "CENTER"]].dropna(axis = 0, how = "any")
weight_center_select.rename(columns = {"ID":"z_sol_id"}, inplace = True)
print(weight_center_select.shape)

(16415, 3)


In [74]:
gengrp6_select_dropna["z_sol_id"] = gengrp6_select_dropna["z_sol_id"].astype(str)
weight_center_select["z_sol_id"] = weight_center_select["z_sol_id"].astype(str)
gengrp6_weight_center = gengrp6_select_dropna.merge(weight_center_select, on = "z_sol_id", how = "outer")
gengrp6_weight_center.dropna(axis=0, subset=["SUBJECT_ID"],how="any", inplace = True)
print(gengrp6_weight_center.shape)
gengrp6_weight_center_dir = os.path.join("..", "raw_data", "phenotype")
gengrp6_weight_center_filename = "gengrp6_center_weight_noex.tsv"
gengrp6_weight_center_dir_filename = os.path.join(gengrp6_weight_center_dir, gengrp6_weight_center_filename)
gengrp6_weight_center.to_csv(gengrp6_weight_center_dir_filename, sep = "\t", header = True, index = False)

(11829, 9)


## Appendix C. Preprocess eGFR

In [129]:
egfr_old_filename = "data_all_2_eGFR_Jan2019.csv"
egfr_old_dir = os.path.join("..", "raw_data", "kidney_pheno")
egfr_old_dir_filename = os.path.join(egfr_old_dir, egfr_old_filename)
egfr_old = pd.read_csv(egfr_old_dir_filename, sep = ",", header = 0, index_col = None)
cat_col = "Study"
egfr_old_summary, category_list = categorize_df(egfr_old, cat_col)
egfr_old_summary_dir = os.path.join("..", "data_summary")
egfr_old_summary_filename = "data_all_2_eGFR_Jan2019_category_summary.tsv"
egfr_old_summary_dir_filename = os.path.join(egfr_old_summary_dir, egfr_old_summary_filename)
egfr_old_summary.to_csv(egfr_old_summary_dir_filename, sep = "\t", header = True, index = False)
display(egfr_old_summary)
save_dir = egfr_old_dir
file_prefix = "eGFR_Jan2019"
split_df(egfr_old, cat_col, category_list, file_prefix, save_dir)

,cohort,sample_size
8,ARIC,3555
9,Amish,1023
0,FHS,3087
5,GENOA,1023
3,GenSalt,1666
7,GeneSTAR,190
2,HyperGEN,1759
4,JHS,3115
6,MESA,4090
1,WHI,4224


1

In [29]:
egfr_old_dir = os.path.join("..", "raw_data", "kidney_pheno", "egfr_freeze5")
egfr_new_dir = os.path.join("..", "raw_data", "phenotype", "egfr_calculated")
cohort_list = ["AMISH", "ARIC", "FHS", "GENOA", "GenSalt", "GeneSTAR", "HyperGEN", "JHS", "MESA", "WHI"]
compare_egfr_df = pd.DataFrame(columns = ["# old not in new", "# new not in old", "# overlap", "# old", "# new", "cor", "# dup old", "# dup new"], index = cohort_list)
for cohort in cohort_list:
    egfr_old_dir_filename = os.path.join(egfr_old_dir, "eGFR_Jan2019_%s.tsv"%cohort)
    egfr_new_dir_filename = os.path.join(egfr_new_dir, "egfr_calculated_%s.tsv"%cohort)
    egfr_old = pd.read_csv(egfr_old_dir_filename, sep = "\t", header = 0, index_col = None)
    egfr_new = pd.read_csv(egfr_new_dir_filename, sep = "\t", header = 0, index_col = None)
    common_col = "id"
    egfr_old_col = "EGFR"
    egfr_new_col = "EGFR"
    egfr_old[egfr_old_col] = egfr_old[egfr_old_col].astype(float)
    egfr_new[egfr_new_col] = egfr_new[egfr_new_col].astype(float)
    df1_nodf2_num, df2_nodf1_num, overlap_num, df1_num, df2_num, cor = compare_df_pair(egfr_old, egfr_new, common_col, egfr_old_col, egfr_new_col)
    compare_egfr_df.loc[cohort, "# old not in new"] = df1_nodf2_num
    compare_egfr_df.loc[cohort, "# new not in old"] = df2_nodf1_num
    compare_egfr_df.loc[cohort, "# overlap"] = overlap_num
    compare_egfr_df.loc[cohort, "# old"] = df1_num
    compare_egfr_df.loc[cohort, "# new"] = df2_num
    compare_egfr_df.loc[cohort, "cor"] = cor
    compare_egfr_df.loc[cohort, "# dup old"] = duplicates_num(egfr_old, common_col)
    compare_egfr_df.loc[cohort, "# dup new"] = duplicates_num(egfr_new, common_col)
    print("Cohort %s completed."%cohort)
display(compare_egfr_df)
compare_egfr_dir = os.path.join("..", "data_summary")
compare_egfr_filename = "compare_egft_calculated_Jan2019_summary_nodup.tsv"
compare_egfr_dir_filename = os.path.join(compare_egfr_dir, compare_egfr_filename)
compare_egfr_df.to_csv(compare_egfr_dir_filename, sep = "\t", header = True, index = True)

Cohort AMISH completed.
Cohort ARIC completed.
Cohort FHS completed.
Cohort GENOA completed.
Cohort GenSalt completed.
Cohort GeneSTAR completed.
Cohort HyperGEN completed.
Cohort JHS completed.
Cohort MESA completed.
Cohort WHI completed.


,# old not in new,# new not in old,# overlap,# old,# new,cor,# dup old,# dup new
AMISH,0,95,1023,1023,1118,1,0,0
ARIC,0,11782,3555,3555,15337,1,0,0
FHS,-52,348,3139,3087,3487,1,0,58
GENOA,0,560,1023,1023,1583,1,0,0
GenSalt,0,180,1666,1666,1846,1,0,0
GeneSTAR,0,16,190,190,206,1,0,0
HyperGEN,0,125,1759,1759,1884,1,0,0
JHS,0,275,3115,3115,3390,1,0,0
MESA,0,2323,4090,4090,6413,1,0,0
WHI,0,468,4224,4224,4692,1,0,0


### **(Deprecated)** Replace the space elements in the MESA egfr table as NAN and remove them

In [170]:
load_dir = os.path.join("..", "raw_data", "phenotype", "egfr_useful")
egfr_new_dir_filename = os.path.join(load_dir, "egfr_calculated_MESA_raw.tsv")
egfr_new = pd.read_csv(egfr_new_dir_filename, sep = "\t", header = 0, index_col = None)
egfr_new.replace(r'^\s*$', np.nan, regex=True, inplace = True)
display(egfr_new.shape)
egfr_new.dropna(axis = 0, how = "any", inplace = True)
display(egfr_new.shape)
egfr_new_mod_dir_filename = os.path.join(load_dir, "egfr_calculated_MESA.tsv")
egfr_new.to_csv(egfr_new_mod_dir_filename, sep = "\t", header = True, index = False)

(6429, 4)

(6413, 4)

### MESA: Correlation between cepgfr1c and eGFR I calculated

In [87]:
freeze5_egfr_dir = os.path.join("..", "raw_data", "kidney_pheno", "egfr_freeze5")
freeze5_egfr_filename = "eGFR_Jan2019_MESA.tsv"
freeze5_egfr_dir_filename = os.path.join(freeze5_egfr_dir, freeze5_egfr_filename)
freeze5_egfr_df = pd.read_csv(freeze5_egfr_dir_filename, sep = "\t", header = 0, index_col = None)
freeze5_egfr = freeze5_egfr_df["EGFR"].values

cepgfr1c_dir = os.path.join("..", "raw_data", "egfr")
cepgfr1c_filename = "TOPMed_kidney_phenotype_MESA_exam1.tsv"
cepgfr1c_dir_filename = os.path.join(cepgfr1c_dir, cepgfr1c_filename)
cepgfr1c_df = pd.read_csv(cepgfr1c_dir_filename, sep = "\t", header = 0, index_col = None)
cepgfr1c = cepgfr1c_df["EGFR"].values

calegfr_dir = os.path.join("..", "raw_data", "egfr")
calegfr_filename = "TOPMed_MESA_RenalPhenotypes_16March2018_egfr.tsv"
calegfr_dir_filename = os.path.join(calegfr_dir, calegfr_filename)
calegfr_df = pd.read_csv(calegfr_dir_filename, sep = "\t", header = 0, index_col = None)
calegfr = calegfr_df["EGFR"].values

calegfr_mod_dir = os.path.join("..", "raw_data", "egfr")
calegfr_mod_filename = "TOPMed_MESA_RenalPhenotypes_16March2018_egfr_mod.tsv"
calegfr_mod_dir_filename = os.path.join(calegfr_mod_dir, calegfr_mod_filename)
calegfr_mod_df = pd.read_csv(calegfr_mod_dir_filename, sep = "\t", header = 0, index_col = None)
calegfr_mod = calegfr_mod_df["EGFR"].values

cor_cepgfr1c_calegfr = pearsonr(cepgfr1c, calegfr)[0]
cor_cepgfr1c_calegfr_mod = pearsonr(cepgfr1c, calegfr_mod)[0]
print(cor_cepgfr1c_calegfr, cor_cepgfr1c_calegfr_mod)

mse_cepgfr1c_calegfr = mean_squared_error(cepgfr1c, calegfr)
mse_cepgfr1c_calegfr_mod = mean_squared_error(cepgfr1c, calegfr_mod)
print(mse_cepgfr1c_calegfr)
print(mse_cepgfr1c_calegfr_mod)

0.9993978212689216 0.9952502226206945
3.394391478720293
33.29065562220411


In [86]:
freeze5_egfr_df.rename(columns = {"EGFR":"EGFR_freeze5"}, inplace = True)
calegfr_df.rename(columns = {"EGFR":"EGFR_calegfr"}, inplace = True)
calegfr_mod_df.rename(columns = {"EGFR":"EGFR_calegfr_mod"}, inplace = True)
egfr_list = [freeze5_egfr_df, calegfr_df, calegfr_mod_df]
merge_egfr = merge_df_list(egfr_list, "id", merge_method='merge', how = 'inner')
common_calegfr = merge_egfr["EGFR_calegfr"].values
common_freeze5_egfr = merge_egfr["EGFR_freeze5"].values
common_calegfr_mod = merge_egfr["EGFR_calegfr_mod"].values

cor_common_freeze5_calegfr = pearsonr(common_freeze5_egfr, common_calegfr)[0]
cor_common_freeze5_calegfr_mod = pearsonr(common_freeze5_egfr, common_calegfr_mod)[0]
print(cor_common_freeze5_calegfr, cor_common_freeze5_calegfr_mod)

mse_common_freeze5_calegfr = mean_squared_error(common_freeze5_egfr, common_calegfr)
mse_common_freeze5_calegfr_mod = mean_squared_error(common_freeze5_egfr, common_calegfr_mod)
print(mse_common_freeze5_calegfr)
print(mse_common_freeze5_calegfr_mod)

0.9959972669338772 0.9999999999999826
17.445292821842674
8.525105559709774e-12


In [85]:
freeze5_egfr_dir = os.path.join("..", "raw_data", "kidney_pheno", "egfr_freeze5")
freeze5_egfr_filename = "eGFR_Jan2019_MESA.tsv"
freeze5_egfr_dir_filename = os.path.join(freeze5_egfr_dir, freeze5_egfr_filename)
freeze5_egfr_df = pd.read_csv(freeze5_egfr_dir_filename, sep = "\t", header = 0, index_col = None)

cepgfr1c_dir = os.path.join("..", "raw_data", "egfr")
cepgfr1c_filename = "TOPMed_kidney_phenotype_MESA_exam1.tsv"
cepgfr1c_dir_filename = os.path.join(cepgfr1c_dir, cepgfr1c_filename)
cepgfr1c_df = pd.read_csv(cepgfr1c_dir_filename, sep = "\t", header = 0, index_col = None)

freeze5_egfr_df.rename(columns = {"EGFR":"EGFR_freeze5"}, inplace = True)
cepgfr1c_df.rename(columns = {"EGFR":"EGFR_cepgfr1c"}, inplace = True)
freeze5_cepgfr1c = freeze5_egfr_df.merge(cepgfr1c_df, left_on = "id", right_on = "sidno", how = "inner")
common_freeze5_egfr = freeze5_cepgfr1c["EGFR_freeze5"].values
common_cepgfr1c = freeze5_cepgfr1c["EGFR_cepgfr1c"].values

cor_common_freeze5_cepgfr1c = pearsonr(common_freeze5_egfr, common_cepgfr1c)[0]
print(cor_common_freeze5_cepgfr1c)

mse_common_freeze5_cepgfr1c = mean_squared_error(common_freeze5_egfr, common_cepgfr1c)
print(mse_common_freeze5_cepgfr1c)

0.9949210599819995
33.930496703190656


### Process DHS Exsited eGFR Table

In [114]:
dhs_dir = os.path.join("..", "raw_data", "kidney_pheno", "dhs")
dhs_filename = "phs001412_AF_20190705_nda.csv"
dhs_dir_filename = os.path.join(dhs_dir, dhs_filename)
dhs = pd.read_csv(dhs_dir_filename, sep = ",", header = 0, index_col = None)
dhs.dropna(axis = 0, subset = ["CKD_eGFR", "Subject_ID", "Study"], inplace = True, how = "any")

check race composition

In [117]:
race_set = set(dhs["Race"].values.tolist())
print(len(race_set), race_set)

1 {'Black'}


In [129]:
dhs.rename(columns = {"Subject_ID":"id", "Sex":"male", "Age":"age", "Race":"race", "CKD_eGFR":"EGFR" }, inplace = True)
race_dict = {"Black":2}
male_dict = {"F":0, "M":1}
dhs.replace({"male":male_dict, "race":race_dict}, inplace = True)

In [132]:
dhs_egfr_sample_num = dhs.shape[0]
dhs_egfr_col_list = ["id", "scr", "race", "age", "male", "EGFR", "case.control"]
dhs_egfr = pd.DataFrame(data = np.zeros((dhs_egfr_sample_num, len(dhs_egfr_col_list))), columns = dhs_egfr_col_list)
dhs_egfr["id"] = dhs["id"]
dhs_egfr["race"] = dhs["race"]
dhs_egfr["age"] = dhs["age"]
dhs_egfr["male"] = dhs["male"]
dhs_egfr["EGFR"] = dhs["EGFR"]
dhs_egfr_dir = os.path.join("..", "raw_data", "egfr")
dhs_egfr_filename = "TopMed_Kidney_Phenotype_DHS_egfr.tsv"
dhs_egfr_dir_filename = os.path.join(dhs_egfr_dir, dhs_egfr_filename)
dhs_egfr.to_csv(dhs_egfr_dir_filename, sep = "\t", header = True, index = False)

### Resolving FHS # old not in new negative issue

In [185]:
load_dir = os.path.join("..", "raw_data", "phenotype", "egfr_useful")
egfr_new_dir_filename = os.path.join(load_dir, "egfr_calculated_FHS_raw.tsv")
egfr_new = pd.read_csv(egfr_new_dir_filename, sep = "\t", header = 0, index_col = None)
dup_num = duplicates_num(egfr_new, "id")
print(dup_num)
boolean_series = egfr_new[["id"]].duplicated(keep=False)
egfr_new_dup = egfr_new.loc[boolean_series, :]
egfr_new_dup.sort_values(axis = 0, by = "id", inplace = True)
#display(egfr_new_dup)
duplicates_list = egfr_new_dup["id"].values.tolist()
print(len(duplicates_list))
print("Total number of all duplicates are two times of number of samples with duplicates, so each sample with duplicate having only one duplicate.")

58
116
Total number of all duplicates are two times of number of samples with duplicates, so each sample with duplicate having only one duplicate.


/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Now I need to check if egfr of each pair of duplicates are identical.

In [187]:
egfr_list = egfr_new_dup["EGFR"].values.tolist()
duplicate_egfr_diff_list = []
egfr_num = len(egfr_list)
for egfr_i in range(0, egfr_num, 2):
    duplicate_egfr_diff = egfr_list[egfr_i + 1] - egfr_list[egfr_i]
    duplicate_egfr_diff_list.append(duplicate_egfr_diff)
print(sum(duplicate_egfr_diff_list))
print("All egfr values of pairs of duplicates are identical, so I will drop any of them in each pair.")

0.0
All egfr values of pairs of duplicates are identical, so I will drop any of them in each pair.


In [188]:
load_dir = os.path.join("..", "raw_data", "phenotype", "egfr_useful")
egfr_new_dir_filename = os.path.join(load_dir, "egfr_calculated_FHS_raw.tsv")
egfr_new = pd.read_csv(egfr_new_dir_filename, sep = "\t", header = 0, index_col = None)
egfr_new.drop_duplicates(subset = "id", keep = "first", inplace = True)
egfr_new_nodup_dir_filename = os.path.join(load_dir, "egfr_calculated_FHS.tsv")
egfr_new.to_csv(egfr_new_nodup_dir_filename, sep = "\t", header = True, index = False)

### Preprocess CHS

CHS SCr file did not include sex info which is needed for calculating CKS-eGFR. So I map the individual ID to the annotation file to get the sex info.

In [47]:
annotation_filename = "freeze8_anno05_af02_unique02.tsv"
annotation_dir = os.path.join("..", "raw_data", "annotation")
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)

chs_dir = os.path.join("..", "raw_data", "kidney_pheno", "chs")
chs_filename = "TOPMed_kidney_phenotype_CHS.txt"
chs_dir_filename = os.path.join(chs_dir, chs_filename)
chs = pd.read_csv(chs_dir_filename, sep = "\t", header = 0, index_col = None)
chs['ID'] = chs['ID'].astype(str)

In [49]:
subject_id_sex = annotation[["subject_id", "sex"]]
subject_id_sex.rename(columns = {"subject_id":"ID", "sex":"MALE"}, inplace = True)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [50]:
chs_sex = chs.merge(subject_id_sex, on = "ID", how = "inner")
chs_sex_filename = "TOPMed_kidney_phenotype_CHS_sex.txt"
chs_sex_dir = chs_dir
chs_sex_dir_filename = os.path.join(chs_sex_dir, chs_sex_filename)
chs_sex.to_csv(chs_sex_dir_filename, sep = "\t", header = True, index = False)

### Preprocess CARDIA

In [31]:
cardia_dir = os.path.join("..", "raw_data", "kidney_pheno", "cardia")
cardia_dcc_filename = "cardia_dcc_demographic_v3.txt"
cardia_scr_filename = "cardia_scr_exam6.txt"
cardia_race_age_filename = "cardia_race_age_exam6.txt"

cardia_dcc_dir_filename = os.path.join(cardia_dir, cardia_dcc_filename)
cardia_scr_dir_filename = os.path.join(cardia_dir, cardia_scr_filename)
cardia_race_age_dir_filename = os.path.join(cardia_dir, cardia_race_age_filename)

cardia_dcc = pd.read_csv(cardia_dcc_dir_filename, sep = "\t", header = 0, index_col = None)
cardia_scr = pd.read_csv(cardia_scr_dir_filename, sep = "\t", header = 0, index_col = None)
cardia_race_age = pd.read_csv(cardia_race_age_dir_filename, sep = "\t", header = 0, index_col = None)

In [32]:
cardia_dcc_scr_race_age_list = [cardia_dcc, cardia_scr, cardia_race_age]
common_col = "ID"
cardia = merge_df_list(cardia_dcc_scr_race_age_list, common_col, merge_method='merge', how = 'inner')

cardia.loc[cardia.loc[:, "MALE"] == "female", "MALE"] = 0
cardia.loc[cardia.loc[:, "MALE"] == "male", "MALE"] = 1

In [33]:
cardia_filename = "cardia_exam6.txt"
cardia_dir_filename = os.path.join(cardia_dir, cardia_filename)
cardia.to_csv(cardia_dir_filename, sep = "\t", header = True, index = False)

### Concatenating All eGFR Tables

In [51]:
egfr_dir = os.path.join("..", "raw_data", "phenotype", "egfr_calculated")
cohort_list = ["AMISH", "ARIC", "CARDIA", "CHS", "DHS", "FHS", "GENOA", "GenSalt", "GeneSTAR", "JHS", "MESA", "WHI", "HCHS-SOL", "HyperGEN"]
header_selection = ["id", "age", "race", "male", "ethnicity", "EGFR"]
egfr_full = concat_egfr(egfr_dir, cohort_list, header_selection)
egfr_full_filename = "egfr03-01.tsv"
egfr_full_dir_filename = os.path.join(egfr_dir, egfr_full_filename)
egfr_full.to_csv(egfr_full_dir_filename, sep = "\t", header = True, index = False)

AMISH completed.
ARIC completed.
CARDIA completed.
CHS completed.


/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


DHS completed.
FHS completed.
GENOA completed.
GenSalt completed.
GeneSTAR completed.
JHS completed.
MESA completed.
WHI completed.
HCHS-SOL completed.
HyperGEN completed.


### Removing and Recording Duplicates in Comprehensive eGFR Table

In [52]:
egfr_dup = df_extraction_duplicates(egfr_full, "unique_subject_key")
print(egfr_dup.shape)
egfr_dup = egfr_dup.sort_values(by=['unique_subject_key'])
egfr_dup_dir = os.path.join("..", "data_summary")
egfr_dup_filename = "egfr03-01_dup.tsv"
egfr_dup_dir_filename = os.path.join(egfr_dup_dir, egfr_dup_filename)
egfr_dup.to_csv(egfr_dup_dir_filename, sep = "\t", header = True, index = False)

(214, 8)


In [53]:
invar_subsets = ["race", "male", "cohort"]
egfr_unique = remove_dup_anno(egfr_full, invar_subsets)
egfr_unique_filename = "egfr03-01_unique.tsv"
egfr_unique_dir_filename = os.path.join(egfr_dir, egfr_unique_filename)
egfr_unique.to_csv(egfr_unique_dir_filename, sep = "\t", header = True, index = False)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Concatenating eGFR to phenotype table

In [30]:
pheno_dir = os.path.join("..", "raw_data", "phenotype")
pheno_filename = "coh03_pre.tsv"
pheno_dir_filename = os.path.join(pheno_dir, pheno_filename)
pheno = pd.read_csv(pheno_dir_filename, sep = "\t", header = 0, index_col = None)
pheno["SUBJECT_ID"] = pheno["SUBJECT_ID"].astype(str)

egfr_dir = os.path.join("..", "raw_data", "phenotype", "egfr_useful")
egfr_filename = "egfr01.tsv"
egfr_dir_filename = os.path.join(egfr_dir, egfr_filename)
egfr = pd.read_csv(egfr_dir_filename, sep = "\t", header = 0, index_col = None)
egfr["SUBJECT_ID"] = egfr["SUBJECT_ID"].astype(str)

pheno_egfr = pheno.merge(egfr, on = ["SUBJECT_ID", "unique_subject_key", "cohort"], how = "outer")
print(pheno_egfr.shape)
print(duplicates_num(pheno_egfr, "unique_subject_key"))
pheno_egfr_filename = "coh03_pheno01_pre.tsv"
pheno_egfr_dir = os.path.join("..", "raw_data", "phenotype")
pheno_egfr_dir_filename = os.path.join(pheno_egfr_dir, pheno_egfr_filename)
pheno_egfr.to_csv(pheno_egfr_dir_filename, sep = "\t", header = True, index = False)

(223838, 37)
0


### (Deprecated) Merging JHS blood cell traits and ddimer with processed results above

In [56]:
pheno_noex_adad_noex_dir = os.path.join("..", "prepro_data", "phenotype")
pheno_noex_adad_noex_filename = "freeze8_anno04_af02_btc02-coh03_ddimer-noex_egfr03-ckd_adad01-noex.tsv"
pheno_noex_adad_noex_dir_filename = os.path.join(pheno_noex_adad_noex_dir, pheno_noex_adad_noex_filename)
pheno_noex_adad_noex = pd.read_csv(pheno_noex_adad_noex_dir_filename, sep = "\t", header = 0, index_col = None)

pheno_dir = os.path.join("..", "raw_data", "phenotype")
jhs_useful_filename = "jhs_usefule_phenotype_05072019.tsv"
jhs_useful_dir_filename = os.path.join(pheno_dir, jhs_useful_filename)
jhs_pheno_useful = pd.read_csv(jhs_useful_dir_filename, sep = "\t", header = 0, index_col = None)

In [57]:
df0_col = "NWDID"
df1_col = "NWDID"
pheno_noex_adad_noex_filljhs = merge_replace_nan(df0_col, df1_col, pheno_noex_adad_noex, jhs_pheno_useful)
pheno_noex_adad_noex_filljhs_dir = os.path.join("..", "prepro_data", "phenotype")
pheno_noex_adad_noex_filljhs_filename = "freeze8_anno04_af02_btc02-coh03_ddimer-noex_egfr03-ckd_adad01-noex_filljhs.tsv"
pheno_noex_adad_noex_filljhs_dir_filename = os.path.join(pheno_noex_adad_noex_filljhs_dir, pheno_noex_adad_noex_filljhs_filename)
pheno_noex_adad_noex_filljhs.to_csv(pheno_noex_adad_noex_filljhs_dir_filename, sep = "\t", header = True, index = False)

## Appendix D. Preprocess CN

### Preprocess CN

In [6]:
cn_dir = os.path.join("..", "raw_data", "cn")
cn_filename = "alpha_globin_calls.txt"
cn_dir_filename = os.path.join(cn_dir, cn_filename)
cn = pd.read_csv(cn_dir_filename, sep = "\t", header = 0, index_col = None)

In [7]:
print(cn.shape)
cn_pass = cn.loc[cn.loc[:, "QC_FAIL"] == "QC_PASS", :]
print(cn_pass.shape)
cn_pass = cn_pass.loc[cn_pass.loc[:, "QC_FLAGGED"] == "QC_PASS", :]
print(cn_pass.shape)

(131823, 8)
(131003, 8)
(130032, 8)


In [ ]:
display(cn_pass)
alpha_globin_calls_pass_useful.tsv

In [431]:
duplicates_num(cn_pass, "SAMPLE")

0

In [9]:
display(cn_pass)

,SAMPLE,QC_FAIL,QC_FLAGGED,CORRECTED_MEAN,CN_CONFIDENCE,CN,CN_HQ,HYBRID_CN
0,NWD100011,QC_PASS,QC_PASS,-0.031077,HIGHER_CONFIDENCE,2,2.0,2.0
1,NWD100014,QC_PASS,QC_PASS,-0.899950,HIGHER_CONFIDENCE,1,1.0,1.0
2,NWD100018,QC_PASS,QC_PASS,-0.268556,HIGHER_CONFIDENCE,2,2.0,2.0
3,NWD100027,QC_PASS,QC_PASS,-0.196768,HIGHER_CONFIDENCE,2,2.0,2.0
4,NWD100028,QC_PASS,QC_PASS,0.066836,HIGHER_CONFIDENCE,2,2.0,2.0
...,...,...,...,...,...,...,...,...
131818,NWD999961,QC_PASS,QC_PASS,0.098204,HIGHER_CONFIDENCE,2,2.0,2.0
131819,NWD999969,QC_PASS,QC_PASS,-0.013932,HIGHER_CONFIDENCE,2,2.0,2.0
131820,NWD999974,QC_PASS,QC_PASS,0.682218,HIGHER_CONFIDENCE,3,3.0,3.0
131821,NWD999977,QC_PASS,QC_PASS,-0.922117,HIGHER_CONFIDENCE,1,1.0,1.0


In [10]:
cn_4 = cn_pass.loc[cn_pass.loc[:, "CN"] == 4, :]

In [11]:
cn_4.shape

(92, 8)

In [20]:
anno_pheno_dir = os.path.join("..", "prepro_data", "phenotype")
anno_pheno_filename = "freeze8_anno05_af02_btc03-coh04_egfr03-ckd_adad01_race_ethnicity_0-2.tsv"
anno_pheno_dir_filename = os.path.join(anno_pheno_dir, anno_pheno_filename)
anno_pheno = pd.read_csv(anno_pheno_dir_filename, sep = "\t", header = 0, index_col = None)

In [23]:
anno_pheno_cn_4 = anno_pheno.merge(cn_4, left_on = "NWDID", right_on = "SAMPLE", how = "inner")
print(anno_pheno_cn_4.shape)

(15, 66)


In [25]:
anno_pheno_cn = anno_pheno.merge(cn_pass, left_on = "NWDID", right_on = "SAMPLE", how = "inner")
print(anno_pheno_cn.shape)

(25233, 66)


In [21]:
display(anno_pheno)

,NWDID,unique_subject_key,consent,exclude,age_at_DDIMER,DDIMER,sample_remove_DDIMER,age_at_hemoglobin_mcnc_bld_1,age_at_hematocrit_vfr_bld_1,age_at_rbc_ncnc_bld_1,...,age_at_CKD,microcytosis,age_at_microcytosis,anemia,age_at_anemia,subject_id,study,sex,ethnicity,AA
0,NWD112649,HCHS_SOL_SoL809397,HMB,False,NaN,NaN,NaN,54.00,54.00,54.00,...,54.0,0.0,54.00,0.0,54.00,SoL809397,HCHS_SOL,0.0,1.0,0.0
1,NWD278543,HCHS_SOL_SoL129930,HMB-NPU,False,NaN,NaN,NaN,22.00,22.00,22.00,...,22.0,0.0,22.00,0.0,22.00,SoL129930,HCHS_SOL,0.0,1.0,0.0
2,NWD579469,HCHS_SOL_SoL379011,HMB-NPU,False,NaN,NaN,NaN,33.00,33.00,33.00,...,33.0,0.0,33.00,0.0,33.00,SoL379011,HCHS_SOL,1.0,1.0,0.0
3,NWD784564,HCHS_SOL_SoL818760,HMB-NPU,False,NaN,NaN,NaN,23.00,23.00,23.00,...,23.0,0.0,23.00,0.0,23.00,SoL818760,HCHS_SOL,0.0,1.0,0.0
4,NWD777183,HCHS_SOL_SoL220081,HMB,False,NaN,NaN,NaN,57.00,57.00,57.00,...,57.0,0.0,57.00,0.0,57.00,SoL220081,HCHS_SOL,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32750,NWD999830,HCHS_SOL_SoL943941,HMB,False,NaN,NaN,NaN,57.00,57.00,57.00,...,57.0,0.0,57.00,0.0,57.00,SoL943941,HCHS_SOL,0.0,1.0,0.0
32751,NWD999882,SAFS_DI00071,DS-DHD-IRB-PUB-MDS-RD,False,NaN,NaN,NaN,47.03,47.03,47.03,...,NaN,0.0,47.03,NaN,47.03,DI00071,SAFS,0.0,1.0,0.0
32752,NWD999922,BioMe_TPMCCDG17054,HMB-NPU,False,NaN,NaN,NaN,72.00,72.00,72.00,...,NaN,0.0,72.00,NaN,72.00,TPMCCDG17054,BioMe,0.0,0.0,1.0
32753,NWD999925,WHI_700450,HMB-IRB-NPU,False,NaN,NaN,NaN,71.07,71.07,NaN,...,71.0,NaN,NaN,1.0,71.07,700450,WHI,0.0,0.0,1.0
